In [1]:
import os
import fnmatch
import pandas as pd
import numpy as np

### Step 1. 
Type in path to the site folder on your computer.

In [2]:
#type in path to site
path = os.path.dirname('/Users/lukefairchild765/Dropbox/Hargraves_Data/Mongolia_Zavkhan/Z21/')

In [3]:
# https://stackoverflow.com/questions/1724693/find-a-file-in-python
def find(pattern, path):
    result = []
    for root, dirs, files in os.walk(path):
        for name in files:
            if fnmatch.fnmatch(name, pattern):
                result.append(os.path.join(root, name))
    return result

#must enter the full extension of the sam file you want here if there are
#multiple sam files--it will choose the default sam file if not
sam_file = find('*.sam', path)

In [4]:
with open(sam_file[0]) as f:
    sample_list = f.readlines()[2:]

In [5]:
samples = []
for sample in [i.strip("\r\n") for i in sample_list]:
    samples.append(find(sample, path)[0])

In [6]:
def check_CSD(sample_list, step, max_CSD):
    data = dict()
    for i in sample_list:
        #sample_ex = pd.read_fwf(i, delim_whitespace=True, skiprows=2, header=None, usecols=[0, 5], names = ['step', 'intensity'])
        sample_ex = pd.read_fwf(i, skiprows=2, widths=[3,3,6,6,6,6,9,6], 
                                names=['demag_type', 'step', 'dec_geo', 'inc_geo', 
                                       'dec_tc', 'inc_tc', 'intensity', 'error_angle'], dtype={'step':str})
        for j in range(len(sample_ex)):
            if sample_ex['step'][j]!='nan':
                sample_ex.demag_type.at[j] = sample_ex['demag_type'][j]+sample_ex['step'][j]
        sample_name = os.path.basename(i)
        data[sample_name] = {}
        # the '.tolist()[-1]' makes sure that you take the last measurement of any one step
        # in the case of duplicates
        data[sample_name]['CSD'] = sample_ex.loc[sample_ex['demag_type']==step]['error_angle'].tolist()
        for err in data[sample_name]['CSD']:
            if err<max_CSD:
                del data[sample_name]
#         data[sample_name]['CSD'] = sample_ex.loc[sample_ex['error_angle']>max_CSD]['error_angle'].tolist()
    return pd.DataFrame(data).transpose()
    return data

#     dataframe = pd.DataFrame(data)
#     #impose a sensible ordering of rows
#     try:
#         dataframe.index = [u'NRM', u'{}'.format(step1), u'{}'.format(step2), u'moment_diff', u'moment_diff_percent']
#     except: #if NRM is one of the steps for comparison
#         dataframe.index = [u'{}'.format(step1), u'{}'.format(step2), u'moment_diff', u'moment_diff_percent']
#     dataframe = dataframe.transpose()
#     return dataframe

### Step 2. 
In the line
```python 
data = check_CSD(samples, 'TT300', 6)
```
replace ```'TT300'``` with the name of the demagnetization step and ```6``` with the maximum CSD of the measurement. The function will return a list of samples for which **all** recorded measurements exceed that CSD.(If there is one measurement that does not exceed ```max_CSD```, that sample will not be returned.)

For example, if you want to check measurements that had just been treated at 100 °C, and you wanted to make sure that all measurements were below a CSD of 8°, you would change the line below to
```python 
data = check_CSD(samples, 'TT100', 8)
```

In [10]:
data = check_CSD(samples,'TT300',6)

### Step 3. 
View and check the output.

In [11]:
data

,CSD
Z21.1a,"[57.3, 61.8, 56.2]"
Z21.2a,"[43.8, 49.5, 37.9]"
Z21.3a,"[20.1, 26.0, 30.9]"
Z21.4a,"[8.8, 9.1, 11.1]"
Z21.6a,"[25.1, 6.9]"


# Step 4. 
Output the data!

In [10]:
# uncomment line below to export the above dataframe to excel
# dataframe.to_excel(os.path.basename(path)+'.xlsx')